In [1]:
import numpy as np
import math
import random
t = 0 #time
Th = [] #collector of theta
num_theta = 0 #num theta
A = [] #collector of arm
num_arms = 0 #number of arms
K = 3 #dimension of arm x, y, theta
Y = [] #collector of arm y
num_arms_y = 0 #number of arms of player two
M = [] #world
num_M = 0
M_vero = 0
theta_vero = 0
theta_stimato = [0, 0, 0, 0, 0, 0, 0, 0, 0] #stima di theta, start at 0,0,...,0
double_arm = [] #collector of double arm
num_double_arm = 0 #number of double arm
R = [] #collection of reward
Id_arm = [] #id of the double arm at time t associated to the reward
played_superarm = [] #number of times each superarm is played
num_best_super_arm = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0]
num_second_best_super_arm = [0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0]
final = [0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0]
sample_correct = [0]*800 #correct at 10, 20, 50, 100, 200, 400, 600, 800
error_reward = [0]*800

delta = 0.05
sigma = 0.1 #noise

def norm_2_mat(x, V):
    b = [x]
    a = np.swapaxes(b, 0, 1)
    ris = np.sqrt(np.matmul(np.matmul(b,V),a))
    return ris[0][0]

class world():
    matrix = np.array([])
    id

    def __init__(self, param):
        self.matrix = param
        global M, num_M
        self.id = num_M
        M.append(self)
        num_M +=1

def create_union_arm(id_x, id_y):
    res = []
    for i in range(0, K):
        for j in range(0, K):
            res.append(A[id_x].arm_vector[i] * Y[id_y].arm_vector[j])
    return res

def create_all_union():
    for i in range(0, num_arms):
        for j in range (0, num_arms_y):
            curr = create_union_arm(i, j)
            double_arm.append(curr)
            global num_double_arm
            num_double_arm += 1
            played_superarm.append(0)


def calcola_A():
    res = 0
    for i in range(0, t):
        #print("i", i)
        if i == 0:
            #print("i==0, print res", res)
            #print(" A[self.A_id[i]].arm_vector",  A[self.A_id[i]].arm_vector)
            res = np.outer(np.transpose(double_arm[Id_arm[i]]), double_arm[Id_arm[i]])
            #print("res", res)
        else:
            #print("i!=0, print i, res", i, res)
            res += np.outer(np.transpose(double_arm[Id_arm[i]]), double_arm[Id_arm[i]])
            #print("A", res)
    res += np.diag([1, 1, 1, 1, 1, 1, 1, 1, 1])
    return res

def calcola_b():
    res = []
    global A
    for i in range(0, t):
        curr = double_arm[Id_arm[i]]
        curr = [R[i] * xx for xx in curr]
        if i == 0:
            res = curr
        else:
            res = [x + y for x, y in zip(res, curr)]
    return res
        
def stima_theta():
    global theta_stimato
    if (t == 0):
        return theta_stimato
    theta_s = np.dot(np.linalg.inv(calcola_A()), calcola_b())
    theta_stimato = theta_s
    return theta_s


def media_reward(index_double_arm):
    return np.dot(double_arm[index_double_arm], np.transpose(stima_theta()))
    
def calcola_bound(index_double_arm):
    ris = 2*sigma*math.sqrt(2)*norm_2_mat(double_arm[index_double_arm], np.linalg.inv(calcola_A()))*math.sqrt(math.log(6/(math.pow(math.pi, 2))*math.pow(t, 2)*K/delta))
    #print("sqrt", math.sqrt(math.log(6/(math.pow(math.pi, 2))*math.pow(self.times, 2)*K/delta)))
    #print("bound", ris)
    return ris
    
def upper_bound(index_double_arm):
    return media_reward(index_double_arm) + calcola_bound(index_double_arm)
    #return self.media_reward(arm_vector)
    
def lower_bound(index_double_arm):
    return media_reward(index_double_arm) - calcola_bound(index_double_arm)
    #return self.media_reward(arm_vector)

class arm():
    arm_vector = np.array([])
    id = 0
    worst_y = -1
    
    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms, A, N
        A.append(self)
        self.id = num_arms
        num_arms +=1        
        
    def generate_double(self):
        for i in range(0, num_arms_y):
            double_arm.append(create_union_arm(self.id, i))

    def pull_arm(self, y_index):
        reward = np.dot(self.arm_vector, np.dot(M[M_vero].matrix, np.transpose(Y[y_index].arm_vector))) + np.random.normal(0, +sigma)
        played_superarm[self.id * num_arms_y + y_index] += 1
        R.append(reward)
        Id_arm.append(self.id * num_arms_y + y_index)
        return reward
    
    def find_worst_y(self):
        #print("find worst y")
        mini = 100000000
        index = - 1
        for i in range(0, num_arms_y):
            curr = np.dot(double_arm[self.id * num_arms_y + i], theta_stimato)
            if curr < mini:
                index = i
                mini = curr
        self.worst_y = index
        #print("mini", mini)
        return mini
    
    
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)
    
class arm_y():
    arm_vector = np.array([])
    id = 0

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms_y, Y
        Y.append(self)
        self.id = num_arms_y
        num_arms_y += 1
        
def run():
    global t
    #print("TIME", t)
    maxi = - 1000
    for i in range(0, num_arms):
        A[i].find_worst_y()
        curr = media_reward(i * num_arms_y + A[i].worst_y)
        #print("curr", curr)
        if curr > maxi:
            best = i
            maxi = curr
        #print("i term and curr value for best, x", i, "y", A[i].worst_y, curr)
    #print("best", best, A[best].worst_y)
    maxi = -1000
    for i in range(0, num_arms):
        if i is not best:
            curr = upper_bound(i * num_arms_y + A[i].worst_y)
            if curr > maxi:
                second_best = i
                maxi = curr
    #print("second best", second_best, A[second_best].worst_y)
    A[best].pull_arm(A[best].worst_y)
    A[second_best].pull_arm(A[second_best].worst_y)
    global num_second_best_super_arm, num_best_super_arm
    num_best_super_arm[best * num_arms_y + A[i].worst_y] +=1
    num_second_best_super_arm[second_best * num_arms_y + A[i].worst_y] +=1
    t+=1
    if (t > 250):
        final[best * num_arms_y + A[i].worst_y] +=1

In [2]:
a = arm( [0.7400797306398084, 0.576547832665651, 0.7349830663582422] )
b = arm( [0.345067231955826, 0.9156681583876762, 0.6823912524346282] )
c = arm( [0.5638993306859795, 0.8748338736453104, 0.18485633496104792] )
d = arm( [0.673719565597578, 0.1725870996908213, 0.7472676319688508] )
m = world( [[0.10491443939828526, 0.6793030919801211, 0.7194757379073787], [0.13546896362398142, 0.1217336458548296, 0.6991816425464136], [0.7898135475171382, 0.21428911809375328, 0.7623852971358278]] )
z = arm_y( [0.7233068388545373, 0.8898790946755905, 0.45671865937762546] )
k = arm_y( [0.36975365980964037, 0.5782920189315391, 0.3785203504783837] )
l = arm_y( [0.8700421843916122, 0.08522705600354452, 0.8195182497977863] )

In [463]:
best_super = -1
best_value = 0

for i in range(0, num_arms):
    for j in range (0, num_arms_y):
        print("x", i, "y", j, "reward", np.dot(A[i].arm_vector, np.dot(M[M_vero].matrix, np.transpose(Y[j].arm_vector))))
        
for i in range(0, num_arms):
    min = 10000
    val_j = 0
    for j in range(0, num_arms_y):
        curr = np.dot(A[i].arm_vector, np.dot(M[M_vero].matrix, np.transpose(Y[j].arm_vector)))
        if curr < min:
            min = curr
            val_j = j
    if min > best_value:
        best_value = min
        best_super = i*(j+1)+val_j
print(best_super)
print(best_value)

x 0 y 0 reward 5.53777694102516
x 0 y 1 reward 4.53219370228344
x 0 y 2 reward 4.278141970381942
x 1 y 0 reward 8.87951362576944
x 1 y 1 reward 5.768104114450812
x 1 y 2 reward 6.503054113551489
x 2 y 0 reward 16.452549428642897
x 2 y 1 reward 11.117449832077849
x 2 y 2 reward 12.458407321460399
x 3 y 0 reward 10.089983645024958
x 3 y 1 reward 6.992810564662383
x 3 y 2 reward 7.623502085805165
7
11.117449832077849


In [464]:
create_union_arm(0,0)

[0.18784601949729962,
 0.014019976659204516,
 0.20395304617371657,
 0.11522305719365272,
 0.008599727461781113,
 0.12510296234641088,
 0.44435909748323343,
 0.033164951760448,
 0.4824610698126172]

In [465]:
create_all_union()

In [466]:
double_arm

[[0.18784601949729962,
  0.014019976659204516,
  0.20395304617371657,
  0.11522305719365272,
  0.008599727461781113,
  0.12510296234641088,
  0.44435909748323343,
  0.033164951760448,
  0.4824610698126172],
 [0.0735589425237237,
  0.1102396979855569,
  0.1531309651396451,
  0.04512039309748307,
  0.06762003826321923,
  0.0939291553881354,
  0.17400733536508495,
  0.26077748591521516,
  0.36223891061565644],
 [0.11944158138522183,
  0.02383181073213605,
  0.16504124032167766,
  0.0732643906965917,
  0.014618218146787473,
  0.10123481095732997,
  0.28254499854903936,
  0.056375333034275166,
  0.3904132586525656],
 [0.5311305850256819,
  0.03964118284208171,
  0.5766728569597037,
  0.04346745310210244,
  0.003244213955437601,
  0.04719460914482887,
  0.18939943497354986,
  0.01413591655922134,
  0.20563966066361894],
 [0.2079863298730028,
  0.31170037800542816,
  0.4329745145402785,
  0.017021493949921963,
  0.025509398149615923,
  0.03543441092603452,
  0.07416724713426986,
  0.111151338

In [467]:
for i in range (0,num_arms):
    for j in range (0, num_arms_y):
        A[i].pull_arm(j)
        t+=1

In [468]:
R

[5.539093736812056,
 4.417574830534825,
 4.170316367804082,
 8.765758423490183,
 5.639868977809668,
 6.483679949039708,
 16.52263265295994,
 11.011328002722022,
 12.608277207365925,
 9.954398602020518,
 7.0837305507858215,
 7.936652080009789]

In [469]:
Id_arm

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [470]:
stima_theta()

array([5.11163671, 1.64475715, 6.75815368, 3.56258983, 1.17077847,
       4.73019171, 1.63063593, 0.65324625, 2.24041729])

In [471]:
for i in range(0,num_arms):
    A[i].find_worst_y()
    print(A[i].worst_y)

1
1
1
1


In [472]:
for i in range(0,num_double_arm):
    print("index", i, "lower_bound", lower_bound(i), "mean", media_reward(i), "upper", upper_bound(i))

index 0 lower_bound 4.823251240370577 mean 5.201094872662228 upper 5.5789385049538796
index 1 lower_bound 3.24398288106753 mean 3.542084014559511 upper 3.840185148051492
index 2 lower_bound 3.618939165167285 mean 3.8943429276166652 upper 4.169746690066045
index 3 lower_bound 7.478784404797062 mean 7.838079428959798 upper 8.197374453122535
index 4 lower_bound 5.004204306381558 mean 5.299511659324387 upper 5.594819012267216
index 5 lower_bound 5.601457854028816 mean 5.861282861686359 upper 6.121107869343902
index 6 lower_bound 13.469429732540112 mean 13.94094906935285 upper 14.412468406165589
index 7 lower_bound 9.004290357469559 mean 9.415355264160896 upper 9.826420170852233
index 8 lower_bound 10.08646904395793 mean 10.423048594746744 upper 10.759628145535558
index 9 lower_bound 8.476002037923633 mean 8.746626670327213 upper 9.017251302730793
index 10 lower_bound 5.673574258974007 mean 5.916389964814192 upper 6.1592056706543765
index 11 lower_bound 6.349418488388356 mean 6.541242751331

In [473]:
run()

In [474]:
for i in range(0,num_double_arm):
    print("index", i, "lower_bound", lower_bound(i), "mean", media_reward(i), "upper", upper_bound(i))

index 0 lower_bound 4.809554461890785 mean 5.19087506123443 upper 5.572195660578076
index 1 lower_bound 3.2972995684266766 mean 3.5968190848306523 upper 3.896338601234628
index 2 lower_bound 3.6206245758584243 mean 3.898580959551138 upper 4.176537343243852
index 3 lower_bound 7.505231544087504 mean 7.867543229038105 upper 8.229854913988706
index 4 lower_bound 5.138513336433262 mean 5.4288518393106955 upper 5.719190342188129
index 5 lower_bound 5.6431501408518585 mean 5.904426789090733 upper 6.165703437329608
index 6 lower_bound 13.659017912685833 mean 14.125148744620498 upper 14.591279576555163
index 7 lower_bound 9.397966772073497 mean 9.76956497524334 upper 10.141163178413182
index 8 lower_bound 10.278949266523686 mean 10.605146200018094 upper 10.931343133512502
index 9 lower_bound 8.55709681040631 mean 8.827009591011864 upper 9.096922371617417
index 10 lower_bound 5.8800408281551295 mean 6.104639609246187 upper 6.329238390337245
index 11 lower_bound 6.438842565948454 mean 6.62720148

In [475]:
R

[5.539093736812056,
 4.417574830534825,
 4.170316367804082,
 8.765758423490183,
 5.639868977809668,
 6.483679949039708,
 16.52263265295994,
 11.011328002722022,
 12.608277207365925,
 9.954398602020518,
 7.0837305507858215,
 7.936652080009789,
 11.206148609232006,
 7.172648934282022]

In [476]:
for i in range (0, 800):
    run()
    if num_best_super_arm.index(max(num_best_super_arm)) == best_super:
        sample_correct[i] += 1
    error_reward[i] += best_value - media_reward(best_super)

In [477]:
sample_correct

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [478]:
num_second_best_super_arm

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1001, 0]

In [479]:
num_best_super_arm

[0, 0, 0, 0, 0, 0, 0, 1001, 0, 0, 0, 0]

In [480]:
for i in range(0,num_double_arm):
    print("index", i, "lower_bound", lower_bound(i), "mean", media_reward(i), "upper", upper_bound(i))

index 0 lower_bound 5.047179832862545 mean 5.496163143064767 upper 5.9451464532669895
index 1 lower_bound 3.9946903594980627 mean 4.211991734537767 upper 4.429293109577471
index 2 lower_bound 3.904708666476061 mean 4.2058040829821906 upper 4.506899499488321
index 3 lower_bound 7.700071082009504 mean 8.179708606958442 upper 8.65934613190738
index 4 lower_bound 5.926842520798409 mean 6.196879890529647 upper 6.466917260260886
index 5 lower_bound 5.916020010196337 mean 6.245379249882856 upper 6.574738489569376
index 6 lower_bound 14.112179548682066 mean 14.704063006917735 upper 15.295946465153405
index 7 lower_bound 11.063151880194512 mean 11.114926649582966 upper 11.166701418971419
index 8 lower_bound 10.852305735045116 mean 11.222211220955073 upper 11.59211670686503
index 9 lower_bound 8.856770532456917 mean 9.212323367897355 upper 9.567876203337793
index 10 lower_bound 6.9319902985677215 mean 6.981946196896722 upper 7.031902095225722
index 11 lower_bound 6.8116871883325185 mean 7.034410

In [481]:
for i in range(0, num_arms):
    for j in range (0, num_arms_y):
        print("x", i, "y", j, "reward", np.dot(A[i].arm_vector, np.dot(M[M_vero].matrix, np.transpose(Y[j].arm_vector))))

x 0 y 0 reward 5.53777694102516
x 0 y 1 reward 4.53219370228344
x 0 y 2 reward 4.278141970381942
x 1 y 0 reward 8.87951362576944
x 1 y 1 reward 5.768104114450812
x 1 y 2 reward 6.503054113551489
x 2 y 0 reward 16.452549428642897
x 2 y 1 reward 11.117449832077849
x 2 y 2 reward 12.458407321460399
x 3 y 0 reward 10.089983645024958
x 3 y 1 reward 6.992810564662383
x 3 y 2 reward 7.623502085805165


In [482]:
final

[0, 0, 0, 0, 0, 0, 0, 763, 0, 0, 0, 0]

In [483]:
pwd

'C:\\Users\\edoar\\PycharmProjects\\LinearBanditExp'

In [484]:
error_reward

[1.243180520334425,
 1.0365656845483855,
 0.9839197364888204,
 0.8688952071614207,
 0.8298184832234092,
 0.7344150687044113,
 0.7051014476986381,
 0.6416299875377689,
 0.6198522598657661,
 0.5732838852319375,
 0.5594813783662858,
 0.5117019268863316,
 0.500669505437715,
 0.45525653112258446,
 0.44448643259284815,
 0.4114058410991177,
 0.4021928397427317,
 0.3762575130265624,
 0.3657882311170262,
 0.34127638879750855,
 0.3300948574361744,
 0.31307173256771037,
 0.30736474575360617,
 0.2890045977418261,
 0.2838003780439866,
 0.26844867788062565,
 0.26333167290881576,
 0.2601639577420709,
 0.25752627330577305,
 0.24300226451773277,
 0.2422554155838288,
 0.2316645315995487,
 0.22911639604529555,
 0.22894571757729487,
 0.2233990799902532,
 0.21409634667281452,
 0.20915208147056674,
 0.20261364962895811,
 0.199908183085876,
 0.19257966005713634,
 0.1882171642558781,
 0.18120041343470206,
 0.18025737771567485,
 0.17540682044252165,
 0.17563421924983125,
 0.16702666119477882,
 0.16630111591166

In [485]:
import numpy as np        
f=open('074034LORO.txt','a')
np.savetxt(f, error_reward, fmt='%1.3f', newline=", ")
f.write("\n")
np.savetxt(f, sample_correct, fmt='%1.3f', newline=", ")
f.write("\n")
f.close()

In [486]:
sample_correct

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
